In [1]:
import pandas as pd
import numpy as np
import torch
import os

from transformers import AutoTokenizer, AutoConfig, TrainingArguments, Trainer
from mkdataset import TypeDataset, EmotionDataset, TimeDataset, ConfidenceDataset, TestDataset
from datasets import load_metric, load_dataset
from classifier import RobertaForSequenceClassification
from sklearn.model_selection import train_test_split

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
backbone = "klue/roberta-base"

In [3]:
seed = 777
os.environ["PYTHONHASHSEED"] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

In [4]:
def compute_metrics(pred):
    f1 = load_metric("f1")
    references = pred.label_ids
    predictions = pred.predictions.argmax(axis=1)
    metric = f1.compute(predictions=predictions, references=references, average="micro")
    return metric

In [5]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [6]:
train_emotion_df, valid_emotion_df = train_test_split(train_df, test_size=0.3, random_state=seed, stratify=train_df.극성)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(backbone)

In [8]:
train_emotion_dataset = EmotionDataset(data=train_emotion_df, tokenizer=tokenizer)
valid_emotion_dataset = EmotionDataset(data=valid_emotion_df, tokenizer=tokenizer)
test_dataset = TestDataset(data=test_df, tokenizer=tokenizer)

In [9]:
gpu = torch.device("cuda:0")

In [10]:
emotion_config = AutoConfig.from_pretrained(backbone)
emotion_config.num_labels = 3

In [11]:
emotion_model = RobertaForSequenceClassification.from_pretrained(backbone, config=emotion_config)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['lstm.bias_hh_l1', 'lstm.bias_hh_l1_reverse', 'roberta.pooler.d

In [12]:
batch_size = 32
save_steps = 90

In [13]:
training_args = TrainingArguments(
    output_dir="./output_type",
    seed=seed,
    save_total_limit=2,
    save_steps = save_steps,
    num_train_epochs = 3,
    learning_rate= 1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,
    weight_decay=1e-4,
    logging_dir="./logs",
    logging_steps = save_steps,
    evaluation_strategy = "steps",
    metric_for_best_model = "eval_f1",
    eval_steps = save_steps,
    load_best_model_at_end=True,
)

In [14]:
emotion_model.to(gpu)

emotion_trainer = Trainer(
    model=emotion_model,
    args=training_args,
    train_dataset=train_emotion_dataset,
    eval_dataset=valid_emotion_dataset,
    compute_metrics=compute_metrics,
)

emotion_trainer.train()

pred_tensor = emotion_trainer.predict(test_dataset)
pred_emotion = pred_tensor.predictions.argmax(axis=1).tolist()

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 11578
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 270


Step,Training Loss,Validation Loss,F1
90,0.152700,0.094490,0.972194
180,0.081900,0.092735,0.970582
270,0.054000,0.089309,0.973605


***** Running Evaluation *****
  Num examples = 4963
  Batch size = 32
Saving model checkpoint to ./output_type/checkpoint-90
Configuration saved in ./output_type/checkpoint-90/config.json
Model weights saved in ./output_type/checkpoint-90/pytorch_model.bin
Deleting older checkpoint [output_type/checkpoint-180] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 4963
  Batch size = 32
Saving model checkpoint to ./output_type/checkpoint-180
Configuration saved in ./output_type/checkpoint-180/config.json
Model weights saved in ./output_type/checkpoint-180/pytorch_model.bin
Deleting older checkpoint [output_type/checkpoint-270] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 4963
  Batch size = 32
Saving model checkpoint to ./output_type/checkpoint-270
Configuration saved in ./output_type/checkpoint-270/config.json
Model weights saved in ./output_type/checkpoint-270/pytorch_model.bin
Deleting older checkpoint [output_type/checkpoint-90

In [15]:
result = pd.DataFrame(pred_emotion, columns=["type"])
test_df_emotion = pd.concat([test_df,result],axis=1, ignore_index=True)
test_df_emotion.to_csv("result_emotion.csv")